## Missing Category Imputation ==> Feature-Engine

Feature-engine is an open source Python package originally designed to support this course, but has increasingly gained popularity and now supports transformations beyond those taught in the course. It was launched in 2017, and since then, several releases have appeared and a growing international community is beginning to lead the development.

- Feature-engine works like to Scikit-learn, so it is easy to learn
- Feature-engine allows you to implement specific engineering steps to specific feature subsets
- Feature-engine can be integrated with the Scikit-learn pipeline allowing for smooth model building
- 
**Feature-Engine allows you to design and store a feature engineering pipeline with different procedures for different variable groups.**

- Make sure you have installed feature-engine before running this notebook.

## In this demo

We will use Feature-engine to perform imputation with a string using the Ames House Price Dataset.

- To download the dataset visit the lecture **Datasets** in **Section 1** of the course.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to split the datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# from feature-engine
from feature_engine.imputation import CategoricalImputer

In [2]:
# let's load the dataset with a selected group of variables

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [3]:
data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

In [4]:
# let's separate into training and testing set

# first drop the target from the feature list
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(
    data[cols_to_use],
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 5), (438, 5))

### Feature-engine captures the categorical variables automatically

In [5]:
# we call the imputer from featur- engine.
# By default it performs imputation with a string missing.

imputer = CategoricalImputer()

In [6]:
# we fit the imputer

imputer.fit(X_train)

CategoricalImputer()

In [7]:
# we see that the imputer found the categorical variables

imputer.variables_

['BsmtQual', 'FireplaceQu']

**This imputer will replace missing data in categorical variables with 'Missing'**

In [8]:
# feature-engine returns a dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,Missing,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,Missing,50.0,0.0,NaN
1384,TA,Missing,60.0,0.0,1939.0
1100,TA,Missing,60.0,0.0,1930.0


In [9]:
# let's check that the numerical variables don't
# contain NA any more

tmp[imputer.variables_].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

## Feature-engine allows you to specify variable groups

In [10]:
# let's impute 1 variable only

imputer = CategoricalImputer(variables=['BsmtQual'])

imputer.fit(X_train)

CategoricalImputer(variables=['BsmtQual'])

In [11]:
# the selected variable

imputer.variables_

['BsmtQual']

In [12]:
# transform data set

tmp = imputer.transform(X_train)

tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,NaN,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,NaN,50.0,0.0,NaN
1384,TA,NaN,60.0,0.0,1939.0
1100,TA,NaN,60.0,0.0,1930.0


In [13]:
tmp[imputer.variables_].isnull().mean()

BsmtQual    0.0
dtype: float64

## Feature-engine can be used with the Scikit-learn pipeline

In [14]:
# let's check the percentage of NA in each categorical variable

X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64

- BsmtQual: 0.023 ==> frequent category imputation
- FirePlaceQu: 0.46 ==> missing category imputation

In [15]:
pipe = Pipeline([
    ('imputer_mode', CategoricalImputer(imputation_method='frequent', variables=['BsmtQual'])),
    ('imputer_missing', CategoricalImputer(variables=['FireplaceQu'])),
])

In [16]:
pipe.fit(X_train)

Pipeline(steps=[('imputer_mode',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['BsmtQual'])),
                ('imputer_missing',
                 CategoricalImputer(variables=['FireplaceQu']))])

In [17]:
pipe.named_steps['imputer_mode'].variables

['BsmtQual']

In [18]:
pipe.named_steps['imputer_missing'].variables

['FireplaceQu']

In [19]:
# let's transform the data with the pipeline
tmp = pipe.transform(X_train)

# let's check null values are gone
tmp.isnull().mean()

BsmtQual       0.000000
FireplaceQu    0.000000
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64